In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
import os #helps with file directories 
import math
import shutil #Move file from one folder to another
import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers.pooling import AvgPool2D
from keras.layers import Conv2D,MaxPool2D,Dropout,Flatten,BatchNormalization,GlobalAvgPool2D
from keras.models import Sequential
from keras.preprocessing.image import load_img,img_to_array,ImageDataGenerator
import keras
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.models import load_model
import numpy as geek

In [ ]:
model=Sequential()
model.add(Conv2D( filters= 16 , kernel_size= (3,3), activation= 'relu', input_shape=(224,224,3) ))

model.add(Conv2D(filters=36,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters=128,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
#currently 2 convolutional layer and 2 maxpool layer
#convolutional layer is the features of the image
#dropping features to avoid overfitting
model.add(Dropout(rate=0.25))

model.add(Flatten())
#dense layers
model.add(keras.layers.Dense(units=64,activation='relu'))
model.add(Dropout(rate=0.25))
#Units=1 gives probability of getting one or zero
model.add(keras.layers.Dense(units=1,activation='sigmoid'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 222, 222, 16)      448       
                                                                 
 conv2d_7 (Conv2D)           (None, 220, 220, 36)      5220      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 36)     0         
 )                                                               
                                                                 
 conv2d_8 (Conv2D)           (None, 108, 108, 64)      20800     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 52, 52, 128)      

In [ ]:
model.compile(optimizer='adam', loss=keras.losses.binary_crossentropy,metrics=['accuracy'])

In [ ]:
def preprocessingimages(path):
  image_data=ImageDataGenerator(zoom_range=0.2,shear_range=0.2,rescale=1/255,horizontal_flip=True)#data augmentation
  image=image_data.flow_from_directory(directory=path,target_size=(224,224),batch_size=32,class_mode='binary')
  return image

In [ ]:
path= "/content/drive/MyDrive/clahe_colon/clahe_colon_new/colon_clahe_train"
train_data=preprocessingimages(path)

Found 7000 images belonging to 2 classes.


In [ ]:
#test
def preprocessingimages2(path):
  image_data=ImageDataGenerator(rescale=1/255)#no augmentation only rescaling
  image=image_data.flow_from_directory(directory=path,target_size=(224,224),batch_size=32,class_mode='binary')
  return image

In [ ]:
path= "/content/drive/MyDrive/clahe_colon/clahe_colon_new/colon_clahe_test"
test_data=preprocessingimages2(path)

Found 1500 images belonging to 2 classes.


In [ ]:
#validation
def preprocessingimages3(path):
  image_data=ImageDataGenerator(rescale=1/255)#no augmentation only rescaling
  image=image_data.flow_from_directory(directory=path,target_size=(224,224),batch_size=32,class_mode='binary')
  return image

In [ ]:
path= "/content/drive/MyDrive/clahe_colon/clahe_colon_new/colon_clahe_val"
val_data=preprocessingimages3(path)

Found 1500 images belonging to 2 classes.


In [ ]:
es=EarlyStopping(monitor="val_accuracy",min_delta=0.01,patience=10,verbose=1,mode='auto')
#patience decides when to stop.Here patience is an issue as it is causing overfitting issues.
#difference between accuracy and validation accuracy greater than 10->Overfitting
#model checkpoint

mc=ModelCheckpoint(monitor="val_accuracy",filepath="./testmodel.h5",verbose=1,save_best_only=True,mode='auto')

cd=[es,mc]

In [ ]:
h5=model.fit_generator(generator=train_data,
                       steps_per_epoch=8,epochs=30,verbose=1,
                       validation_data=val_data,validation_steps=16,
                       callbacks=cd)

VGG


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Flatten,Dense
from keras.models import Model,load_model
from keras.applications.vgg16 import VGG16,preprocess_input

In [ ]:
base_model=VGG16(input_shape=(224,224,3),include_top=False)


58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
for layer in base_model.layers:
  layer.trainable=False

base_model.summary

<bound method Model.summary of <keras.engine.functional.Functional object at 0x7f84ddff7950>>

In [ ]:
#Modifying preprocessing on test data suitable for mobilenet
def preprocessingimages4(path):
  image_data=ImageDataGenerator(preprocessing_function=preprocess_input)#no augmentation only rescaling
  image=image_data.flow_from_directory(directory=path,target_size=(224,224),batch_size=32,class_mode='binary')
  return image

In [ ]:
path= "/content/drive/MyDrive/clahe_colon/clahe_colon_new/colon_clahe_test"
ctest_data=preprocessingimages2(path)

Found 1500 images belonging to 2 classes.


In [ ]:
def preprocessingimages5(path):
  image_data=ImageDataGenerator(preprocessing_function=preprocess_input)#no augmentation only rescaling
  image=image_data.flow_from_directory(directory=path,target_size=(224,224),batch_size=32,class_mode='binary')
  return image

In [ ]:
path= "/content/drive/MyDrive/clahe_colon/clahe_colon_new/colon_clahe_val"
cval_data=preprocessingimages3(path)

Found 1500 images belonging to 2 classes.


In [ ]:
def preprocessingimages6(path):
  image_data=ImageDataGenerator(zoom_range=0.2,shear_range=0.2,preprocessing_function=preprocess_input,horizontal_flip=True)#data augmentation
  image=image_data.flow_from_directory(directory=path,target_size=(224,224),batch_size=32,class_mode='binary')
  return image

In [ ]:
path= "/content/drive/MyDrive/clahe_colon/clahe_colon_new/colon_clahe_train"
ctrain_data=preprocessingimages(path)

Found 7000 images belonging to 2 classes.


In [ ]:
X=Flatten()(base_model.output)
X=Dense(units=1,activation='sigmoid')(X)

model=Model(base_model.input,X)

model.summary

<bound method Model.summary of <keras.engine.functional.Functional object at 0x7f84ddfcdad0>>

In [ ]:
model.compile(optimizer='adam',loss=keras.losses.binary_crossentropy,metrics=['accuracy'])

In [ ]:
#callbacks
from keras.callbacks import ModelCheckpoint,EarlyStopping 
mc=ModelCheckpoint(filepath="newmodel.h5",monitor="val_accuracy",verbose=1,save_best_only=True,mode='auto')
es=EarlyStopping(monitor="val_accuracy",min_delta=0.01,patience=3,verbose=1)
cb=[mc,es]


In [ ]:
hist=model.fit_generator(ctrain_data,steps_per_epoch=8,
                         epochs=30,validation_steps=16,
                         validation_data=cval_data,callbacks=cb)

Epoch 1/30
8/8 [==============================] - ETA: 0s - loss: 0.8115 - accuracy: 0.5742 
Epoch 1: val_accuracy improved from -inf to 0.79688, saving model to newmodel.h5
8/8 [==============================] - 358s 48s/step - loss: 0.8115 - accuracy: 0.5742 - val_loss: 0.4148 - val_accuracy: 0.7969
Epoch 2/30
8/8 [==============================] - ETA: 0s - loss: 0.3479 - accuracy: 0.8594 
Epoch 2: val_accuracy improved from 0.79688 to 0.91211, saving model to newmodel.h5
8/8 [==============================] - 353s 47s/step - loss: 0.3479 - accuracy: 0.8594 - val_loss: 0.2413 - val_accuracy: 0.9121
Epoch 3/30
8/8 [==============================] - ETA: 0s - loss: 0.2367 - accuracy: 0.9180 
Epoch 3: val_accuracy improved from 0.91211 to 0.95898, saving model to newmodel.h5
8/8 [==============================] - 352s 47s/step - loss: 0.2367 - accuracy: 0.9180 - val_loss: 0.1357 - val_accuracy: 0.9590
Epoch 4/30
8/8 [==============================] - ETA: 0s - loss: 0.1538 - accuracy: 

In [ ]:
from keras.models import load_model
model=load_model("/content/newmodel.h5")

In [ ]:
acc=model.evaluate_generator(ctest_data)[1]#i is given as it returns loss and accuracy
print(acc)

0.9833333492279053


In [ ]:
path="/content/drive/MyDrive/clahe_colon/clahe_colon_new/colon_clahe_test/colon_clahe_aca/4252.jpg"
img=load_img(path,target_size=(224,224))
input_arr=img_to_array(img)/255#dividing the imag by 255 to normalise the image
input_arr.shape
input_arr=np.expand_dims(input_arr,axis=0)

#pred=model.predict_classes(input_arr)[0][0]
#pred
pred=(model.predict(input_arr)[0][0] > 0.5).astype("int32")
if pred:
  print("Benign")
else:
  print("Malignant") 

Malignant
